In [2]:
import torch
import numpy as np
import cv2
from facenet_pytorch import InceptionResnetV1

# Load FaceNet model and embeddings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionResnetV1(pretrained='vggface2').to(device).eval()
face_embeddings = np.load("face_embeddings.npy", allow_pickle=True).item()

# Load test image
img_path = r"C:\Code\Alan Guedes - CSMPR21_data\test_reference_images\Test_Ref_01.jpg"
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (160, 160)) / 255.0
img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)

# Extract features
with torch.no_grad():
    test_embedding = model(img).cpu().numpy().flatten()

# Compare with stored embeddings
best_match = None
best_distance = float("inf")
for person, embedding in face_embeddings.items():
    distance = np.linalg.norm(test_embedding - embedding)
    if distance < best_distance:
        best_distance = distance
        best_match = person

# Display result
if best_distance < 1.2:  # Adjust threshold if needed
    print(f"Recognized: {best_match} (distance: {best_distance:.2f})")
else:
    print("No match found!")

Recognized: Ref_1072 (distance: 1.13)
